In [1]:
import pandas as pd
import yfinance as yf
from datetime import date

In [2]:
symbol = 'SNOW'
calendar_days = 191

start_date = '2023-1-1'
trading_days = round(calendar_days * 252 / 365)
sma_days = trading_days
tickerData = yf.Ticker(symbol)
history = tickerData.history(interval='1d', start=start_date, end=date.today())
history['SMA'] = history['Close'].rolling(window=sma_days).mean()
history['SMA_dist'] = history.Close / history.SMA - 1

In [5]:
last_sma = history.SMA.iloc[-1]
last_price = history.Close.iloc[-1]

sma_dist = history.SMA_dist.dropna()
sma_range = (sma_dist <= sma_dist.iloc[-1]).mean()

print(symbol)
print(f'Start:   {history.index[0].date()}')
print(f'Price:   ${last_price:.2f} (P{sma_range:.0%})')
print(f'SMA {sma_days:3d}d: ${last_sma:.2f}')

for q in [0.5, 0.25, 0.1, 0.05]:
  dist = sma_dist.quantile(q)
  price = last_sma * (1 + dist)
  print(f'P   {q:4.0%}: ${price:5.2f} (SMA{sma_days}{dist:+.1%})')


SNOW
Start:   2023-01-03
Price:   $182.88 (P96%)
SMA 132d: $136.35
P    50%: $127.66 (SMA132-6.4%)
P    25%: $116.02 (SMA132-14.9%)
P    10%: $106.59 (SMA132-21.8%)
P     5%: $102.95 (SMA132-24.5%)
